# 문제 연설문을 기계학습하기 위해 변환

## pyspark 기본 셋팅

In [1]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 함수 정의

In [2]:
def normDist(x1, x2):
    dist = np.linalg.norm(x1 - x2)
    return dist

In [7]:
def cluster(x, k, centroids):
    row, col = x.shape
    clusterAssigned = np.zeros(shape=(row, k), dtype=float)
    clusterUpdated=True
    iter = 1
    while(clusterUpdated):
        print ("\n\n====== Iteration {} ======".format(iter))
        for i in range(row):
            minDist = np.inf   
            minIndex = -1      
            for j in range(k):
                distJI = normDist(centroids[j], x[i])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            print (".....Assigning {} to {}th centroid by {:.3f}".format(x[i], minIndex, minDist))
            clusterAssigned[i] = [minIndex, minDist]
        print ("\n.....Assigned clusters: \n {}".format(clusterAssigned))
        _centroids=np.zeros(shape = (k, x.shape[1]), dtype=float)
        for cent in range(k):
            _centroids[cent] = np.mean(x[clusterAssigned[:,0]==cent],axis=0)
        comparison = np.equal(centroids, _centroids)
        if(comparison.all() == True):
            print ("\n.....NOT Updating Centroids\n {} => \n{}".format(centroids, _centroids))
            clusterUpdated = False
            return centroids, clusterAssigned
        else:
            print ("\n.....Updating Centroids\n {} => \n{}".format(centroids, _centroids))
            centroids = _centroids
            iter += 1

In [4]:
def draw(x, centroids):
    import matplotlib.pyplot as plt
    plt.scatter(x[:,0], x[:,1], color='blue')
    plt.scatter(centroids[:,0], centroids[:,1], color='red')

### 클러스터링 실행

In [12]:
from sklearn.datasets import make_blobs
import numpy as np

X, y = make_blobs(n_samples=100, centers=4, cluster_std=0.60, random_state=0)
import matplotlib.pyplot as plt
X, y

(array([[ 6.69786996e-01,  3.59540802e+00],
        [-2.02396992e+00,  7.99569054e+00],
        [ 1.43289271e+00,  4.37679234e+00],
        [ 1.06923853e+00,  4.53068484e+00],
        [ 1.71444449e+00,  5.02521524e+00],
        [ 7.43873988e-01,  4.12240568e+00],
        [-2.04363942e+00,  4.06392123e+00],
        [-6.78603291e-01,  7.88799076e+00],
        [-4.74920358e-02,  5.47425256e+00],
        [-1.46976488e+00,  7.69183251e+00],
        [-8.64176859e-01,  6.86528639e+00],
        [-1.84838498e+00,  6.90859736e+00],
        [-1.27182547e+00,  7.13460392e+00],
        [-9.34289778e-01,  7.73253222e+00],
        [-7.34157408e-01,  7.44484466e+00],
        [ 3.18515794e+00,  8.90082233e-02],
        [ 1.35139348e+00,  2.06383637e+00],
        [-9.70171483e-02,  3.48456995e+00],
        [-1.93150361e+00,  2.93698060e+00],
        [ 2.73890793e+00,  1.56768168e-01],
        [ 2.33812285e+00,  3.43116792e+00],
        [-1.93673697e+00,  7.57276799e+00],
        [-1.98885366e+00,  3.241